In [ ]:
import numpy as np
import pandas as pd

import georges

from georges import manzoni
from georges.manzoni import *

from georges import Kinematics
from georges import ureg as _ureg

### Kinematics object

In [ ]:
k = Kinematics(70*_ureg.MeV) # As 70 is lower than the proton mass energy, the object recognize it as kinetic energy
k

In [ ]:
k = Kinematics(1000*_ureg.MeV) # Now, as 1000 is higher than the proton mass energy, it is set as the total energy
k

In [ ]:
k = Kinematics(1000*_ureg.MeV, kinetic=True) # If you want 1000 MeV (or any energy higher than 938.272 MeV)
                                             # to be the kinetic energy, set the keyword "kinetic" to True
k

In [ ]:
k.range # We can easily extract any of the computed kinematics parameters

In [ ]:
k.range.to('meter') # We can convert any parameter to a valid pint system unit using the ".to()" method

In [ ]:
type(k.range.to)

We can compute the kinematics starting from any other physical parameter

In [ ]:
Kinematics(0.36) ## From beta

In [ ]:
Kinematics(130*_ureg.MeV_c) ## From momentum

### Definition of a FODO beamline

In [ ]:
FODO = [Marker(LABEL1='M'),
        Drift(LABEL1 = 'D1', L = 0.3*_ureg.meter),
        Quadrupole(LABEL1 = 'Q1', L = 0.2*_ureg.meter, K1 = 60/_ureg.meter**2,
                    APERTYPE='CIRCULAR', APERTURE=[0.005*_ureg.m]), 
        Drift(LABEL1 = 'D2', L = 0.3*_ureg.meter,
                    APERTYPE='CIRCULAR', APERTURE=[0.005*_ureg.m]),
        Quadrupole(LABEL1 = 'Q2', L = 0.2*_ureg.meter, K1 = -60/_ureg.meter**2,
                    APERTYPE='CIRCULAR', APERTURE=[0.005*_ureg.m]),
        Drift(LABEL1 = 'D3', L = 0.3*_ureg.meter,
                    APERTYPE='CIRCULAR', APERTURE=[0.005*_ureg.m])]
FODO

### Generate a MANZONI input object

Pass the beamline as the "sequence" parameter to the manzoni.Input object 

In [ ]:
fodo_beamline = manzoni.Input(sequence=FODO) 
fodo_beamline.sequence

### Generate the beam

In [ ]:
energy     = 230*_ureg.MeV
kin      = Kinematics(energy)
covariance = np.eye(6)*0.005**2
covariance[1,1] = 0.005**2
covariance[3,3] = 0.005**2
print(covariance)
beam_distr = np.random.multivariate_normal(np.zeros(6), covariance, int(1e5))
print(beam_distr)
beam       = Beam(kinematics=kin, distribution=beam_distr)

### Definition of some observers and tracking along the line

Here we only define the beam and sigma observers, but others exist, and you can also define yours !

In [ ]:
# %%timeit -n 1 -r 1
sigma_obs = SigmaObserver()
beam_obs = BeamObserver(with_input_beams=True)
track(beamline=fodo_beamline, beam=beam, observers=[beam_obs, sigma_obs])

### Show the observers results

In [ ]:
beam_obs.to_df()

In [ ]:
sigma_obs.to_df()

### Position where to observe the results

We can decide were we want to observe the results. For that we use the "elements" parameter of the observer

In [ ]:
sigma_obs = SigmaObserver(elements=['Q1', 'Q2']) # Here we observe the beam sigmas only for the quadrupoles
track(beamline=fodo_beamline, beam=beam, observers=[sigma_obs])
sigma_obs.to_df()

## Plot of the tracking results

We need to generate a dataframe based on our sequence first

In [ ]:
fodo_df = pd.DataFrame({'NAME': ['M','D1', 'Q1', 'D2', 'Q2', 'D3'],
                        'TYPE': ['Marker','Drift', 'Quadrupole', 'Drift', 'Quadrupole', 'Drift'],
                        'CLASS': ['Marker','Drift', 'Quadrupole', 'Drift', 'Quadrupole', 'Drift'],
                        'L': [0.0, 
                              fodo_beamline.sequence[1].L.m_as('m'),
                              fodo_beamline.sequence[2].L.m_as('m'),
                              fodo_beamline.sequence[3].L.m_as('m'),
                              fodo_beamline.sequence[4].L.m_as('m'),
                              fodo_beamline.sequence[5].L.m_as('m')]})

fodo_df['AT_ENTRY'] = 0.0
fodo_df['AT_CENTER'] = 0.0
fodo_df['AT_EXIT'] = 0.0
fodo_df['APERTYPE'] = 'Circular'
fodo_df['APERTURE'] = [[0.0, 0.0]] * len(fodo_df)

for idx, line in fodo_df.iterrows():
    if idx == 0.0:
        fodo_df.at[idx,'AT_ENTRY'] = 0.0
        fodo_df.at[idx,'AT_CENTER'] = fodo_df.at[idx, 'L'] / 2
        fodo_df.at[idx,'AT_EXIT'] =fodo_df.at[idx,'L']
        
    else:
        fodo_df.at[idx,'AT_ENTRY'] = fodo_df.at[idx-1,'AT_ENTRY'] + fodo_df.at[idx-1,'L']
        fodo_df.at[idx,'AT_CENTER'] = fodo_df.at[idx,'AT_ENTRY'] + fodo_df.at[idx,'L'] /2 
        fodo_df.at[idx,'AT_EXIT'] = fodo_df.at[idx,'AT_ENTRY'] + fodo_df.at[idx,'L']
        
    fodo_df.at[idx,'APERTURE'] = [0.05*_ureg.m, 0.05*_ureg.m]

fodo_df.set_index('NAME', inplace=True)
fodo_df['L'] = fodo_df['L'].apply(lambda e: e*_ureg.m)
fodo_df

The beam results dataframe must be indexed

In [ ]:
beam_o_df = beam_obs.to_df()
beam_o_df.set_index('LABEL1', inplace=True)

In [ ]:
from georges import vis
import matplotlib.pyplot as plt

plt.rc('text', usetex=False)
fig = plt.figure(figsize=(15,10))

ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
manzoni_plot = vis.ManzoniMatplotlibArtist(ax=ax)
manzoni_plot2 = vis.ManzoniMatplotlibArtist(ax=ax2)

manzoni_plot.prepare(ax, fodo_df, with_beamline=False, print_label=True) # Preparation of the plot
manzoni_plot.aperture(ax, fodo_df, plane='X') # To visualize the aperture of the beamline  elements
manzoni_plot.tracking(ax, fodo_df, beam_o_df, plane='X',halo_99=True) # Plot the beam sizes in the defined 
                                                                            # plane, at the exit of each element

manzoni_plot2.prepare(ax2, fodo_df, with_beamline=False, print_label=True)
manzoni_plot2.aperture(ax2, fodo_df, plane='Y')
manzoni_plot2.tracking(ax2, fodo_df, beam_o_df, plane='Y',halo_99=True)

In [ ]:
fig = manzoni_plot.summary(bl=fodo_df, beam_o_df=beam_o_df, element='D1') # A summary of results at the exit of D1
                                                                            # (beam profile, losses, transmission)

### Selection of another manzoni integrator 

By default the tracking uses the MadXIntegrator maps

To select any other integrator, change the "integrator" method of the beamline elements

In [ ]:
fodo_beamline.sequence[0].integrator

In [ ]:
fodo_beamline.sequence[0].integrator = georges.manzoni.integrators.Mad8FirstOrderTaylorIntegrator # Here we change the integrator of the first drift
print(fodo_beamline.sequence[0].integrator)
print(fodo_beamline.sequence[1].integrator)

### NOTE: Be careful about the required particle coordinates for each integrator !

### Adjustment of the input beam energy for degrader type elements

This feature is useful for degraders

Here both the manzoni and the fermi modules of the georges library are involved

In [ ]:
from georges.fermi import materials as gfmaterials

thick_2 = gfmaterials.Lexan.required_thickness(kinetic_energy_in=230*_ureg.MeV,
                                    kinetic_energy_out=179.3*_ureg.MeV)
degrader_beamline = manzoni.Input(sequence=[
    Drift(LABEL1 = 'D1', L = 0.3*_ureg.meter),
    Degrader(LABEL1 = 'Deg1', L = 0.10*_ureg.meter, MATERIAL=georges.fermi.materials.Lexan),
    Drift(LABEL1 = 'D2', L = 0.3*_ureg.meter),
    Degrader(LABEL1 = 'Deg2', L = thick_2, MATERIAL=georges.fermi.materials.Lexan),
    Drift(LABEL1 = 'D3', L = 0.3*_ureg.meter),
    Degrader(LABEL1 = 'Deg3', L = 0.05*_ureg.meter, MATERIAL=georges.fermi.materials.Water),
    Drift(LABEL1 = 'D4', L = 0.3*_ureg.meter)
])

degrader_beamline.sequence

In [ ]:
degrader_beamline.adjust_energy(input_energy=230*_ureg.MeV)
degrader_beamline.sequence

### The freeze function for degraders elements

This is to avoid a repetitive computation of the degraders fermi-eyges integrals 

This helps to save time if we need to re optimize for example the optics of the line

In [ ]:
%%timeit -n 1 -r 1
degrader_beamline.freeze()

In [ ]:
%%timeit -n 1 -r 1
sigma_obs = SigmaObserver()
track(beamline=degrader_beamline, beam=beam, observers=[sigma_obs])

### Make a quadrupole subclass manually

In [ ]:
import georges_core.sequences
georges_core.sequences.elements.Element.Quadrupole.make_subclass('QSHORT',
                                                                 L=10 * _ureg.cm)

q1g = georges_core.sequences.elements.Element.Quadrupole.QSHORT()
q1g